## Black-Scholes Formulas
### d1 and d2
#### $ d_1 = \frac{ln(\frac{S}{K}) + (r - q + \frac{\sigma^2}{2})*T}{\sigma\sqrt{T}} $
#### $ d_2 = d_1 - \sigma\sqrt{T} $

### Option Price
#### For Call Options: $ C = Se^{-qT}N(d_1) - Ke^{-rT}N(d_2) $ 
#### For Put Options: $ P = Ke^{-rT}N(-d_2) - Se^{-qT}N(-d_1) $

## Greeks
### Delta($\Delta = \frac{\delta V}{\delta S}$): 
#### For Call Options: $ \Delta_c = e^{-qT}N(d_1) $ 
#### For Put Options: $ \Delta_p = -e^{-qT}N(-d_1) $

### Gamma($\Gamma = \frac{\delta^2 V}{\delta S^2}$):
#### $ \Gamma = \frac{e^{-qT}N'(d_1)}{S\sigma\sqrt{T}} $

### Vega($\nu = \frac{\delta V}{\delta \sigma}$):
#### $ \nu = 0.01 * Se^{-qT}N'(d_1)\sqrt{T} $

### Theta($\Theta = \frac{\delta V}{\delta T}$):
#### For Call Options: $ \theta_c = \frac{1}{252} * \left(-\frac{Se^{-qT}N'(d_1)\sigma}{2\sqrt{T}} + qSe^{-qT}N(d_1) - rKe^{-rT}N(d_2)\right) $
#### For Put Options: $ \theta_p = \frac{1}{252} * \left(-\frac{Se^{-qT}N'(d_1)\sigma}{2\sqrt{T}} - qSe^{-qT}N(-d_1) + rKe^{-rT}N(-d_2)\right) $

### Rho($\rho = \frac{\delta V}{\delta r}$):
#### For Call Options: $ \rho_c = 0.01 * KTe^{-rT}N(d_2) $
#### For Put Options: $ \rho_p = -0.01 * KTe^{-rT}N(-d_2) $

#### N(d) is the cumulative distribution function (cdf) of standard normal distribution,
#### N'(d) is the probability density function (pdf) of standard normal distribution,
#### Theta is expressed as the daily rate of time decay, so we divided by 252 to account for number of trading days per year.

In [274]:
import numpy as np
from scipy.stats import norm
import yfinance as yf
import pandas as pd

In [275]:
def BlackScholes(r, S, K, T, sigma, option_type="C", q=0):
    """
Calculate the price of a European Call or Put option using the Black-Scholes formula, including adjustments for continuous dividend yield.

Parameters
----------
r : float
    Risk-free interest rate, expressed as a decimal (e.g., 0.05 for 5%).
S : float
    Current stock price.
K : float
    Strike price of the option.
T : float
    Time to expiration of the option, expressed in years (e.g., 0.5 for six months).
sigma : float
    Volatility of the underlying stock, expressed as a decimal.
option_type : str, optional
    Type of the option: 'C' for Call option or 'P' for Put option. The default is 'C'.
q : float, optional
    Continuous dividend yield of the underlying stock, expressed as a decimal. The default is 0.

Returns
-------
None
    This function prints the option price, Delta, Gamma, Vega, Theta, and Rho for the specified Call or Put option.
    - Option Price: The theoretical price of the option using the Black-Scholes formula.
    - Delta: Measures the rate of change of the option's price with respect to changes in the underlying asset's price.
    - Gamma: Measures the rate of change in Delta with respect to changes in the underlying asset's price.
    - Vega: Measures sensitivity of the option's price to changes in the volatility of the underlying asset. Note that Vega is not a Greek letter; the symbol ν (nu) is sometimes used.
    - Theta: Measures the rate of time decay of the option's price, expressed per day. Used 252 trading days.
    - Rho: Measures sensitivity of the option's price to changes in the risk-free interest rate.

Note
----
This function assumes European options, which can only be exercised at expiration. It does not apply to American options, which can be exercised at any time before expiration.
----
"""
    
    # Define d1 and d2 for Black-Scholes, including the dividend yield
    d1 = (np.log(S / K) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Subfunction to calculate delta
    def calc_delta():
        if option_type == "C":
            return norm.cdf(d1, 0, 1)  # Delta for Call option
        elif option_type == "P":
            return -norm.cdf(-d1, 0, 1)  # Delta for Put option
        
    # Subfunction to calculate Gamma
    def calc_gamma():
        return norm.pdf(d1, 0, 1) / (S * sigma * np.sqrt(T))
    
    # Subfunction to calculate Vega
    def calc_vega():
        return (S * norm.pdf(d1, 0, 1) * np.sqrt(T)) * 0.01
    
    # Subfunction to calculate Theta, including dividend yield
    def calc_theta():
        if option_type == "C":
            return (-S * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) + q * S * norm.cdf(d1, 0, 1) - r * K * np.exp(-r * T) * norm.cdf(d2, 0, 1)) / 252 # Theta for Call option
        elif option_type == "P":
            return (-S * norm.pdf(d1, 0, 1) * sigma / (2 * np.sqrt(T)) - q * S * norm.cdf(-d1, 0, 1) + r * K * np.exp(-r * T) * norm.cdf(-d2, 0, 1)) / 252 # Theta for Put option

    # Subfunction to calculate Rho
    def calc_rho():
        if option_type == "C":
            return (K * T * np.exp(-r * T) * norm.cdf(d2, 0, 1)) * 0.01  # Rho for Call option
        elif option_type == "P":
            return (-K * T * np.exp(-r * T) * norm.cdf(-d2, 0, 1)) * 0.01  # Rho for Put option

    try:
        delta = calc_delta()  # Call the subfunction to calculate delta
        gamma = calc_gamma()  # Call the subfunction to calculate gamma
        vega = calc_vega()    # Call the subfunction to calculate vega
        theta = calc_theta()  # Call the subfunction to calculate theta
        rho = calc_rho()      # Call the subfunction to calculate rho
        if option_type == "C":
            price = S * np.exp(-q * T) * norm.cdf(d1, 0, 1) - K * np.exp(-r * T) * norm.cdf(d2, 0, 1)
            print(f"Call Option Price: {price:.5f}")
            print("-----------------")
            print(f"Call Delta: {delta:.5f}")
            print(f"Gamma: {gamma:.5f}")
            print(f"Vega: {vega:.5f}")
            print(f"Call Theta: {theta:.5f}")
            print(f"Call Rho: {rho:.5f}")
        elif option_type == "P":
            price = K * np.exp(-r * T) * norm.cdf(-d2, 0, 1) - S * np.exp(-q * T) * norm.cdf(-d1, 0, 1)
            print(f"Put Option Price: {price:.5f}")
            print("-----------------")
            print(f"Put Delta: {delta:.5f}")
            print(f"Gamma: {gamma:.5f}")
            print(f"Vega: {vega:.5f}")
            print(f"Put Theta: {theta:.5f}")
            print(f"Put Rho: {rho:.5f}")
        else:
            raise ValueError("Invalid option type! Please confirm option type either 'C' for Call option or 'P' for Put option")
    except ValueError as e:
        print(e)

In [296]:
BlackScholes(0.05, 50, 48, 0.5, 0.4, "C", 0.02)

Call Option Price: 6.86428
-----------------
Call Delta: 0.63261
Gamma: 0.02664
Vega: 0.13318
Call Theta: -0.02348
Call Rho: 0.12226


#### Let's try this on a USD based FX pair, "USDCHF=X"

In [284]:
# Download price data from yfinance for 1-year and calculate historical volatility
data = yf.download("USDCHF=X", start="2018-02-21", end="2024-02-21")
daily_rets = data['Adj Close'].pct_change().dropna()
current_price = data['Adj Close'][-1]

#Risk-free rate will be obtained from US 10-Y Treasury Bonds "^TNX"
rf = yf.download("^TNX", start="2018-02-21", end="2024-02-21")['Adj Close'][-1] / 100

[*********************100%***********************]  1 of 1 completed


C:\Users\emir.e\AppData\Local\Temp\ipykernel_20780\1215793644.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = data['Adj Close'][-1]


[*********************100%***********************]  1 of 1 completed


C:\Users\emir.e\AppData\Local\Temp\ipykernel_20780\1215793644.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rf = yf.download("^TNX", start="2018-02-21", end="2024-02-21")['Adj Close'][-1] / 100


In [285]:
BlackScholes(rf, current_price, 0.82, 0.25, 0.4, "P")

Put Option Price: 0.03764
-----------------
Put Delta: -0.30080
Gamma: 1.97174
Vega: 0.00154
Put Theta: -0.00044
Put Rho: -0.00076


In [286]:
# Now let's try one on a stock with an assumed dividend yield of 2%, using "AKBNK.IS"
data = yf.download("AKBNK.IS", start="2023-02-21", end="2024-02-21")
daily_rets = data['Adj Close'].pct_change().dropna()
current_price = data['Adj Close'][-1]

# We will have to obtain the risk-free rate by Webscraping
rf = pd.read_html("https://www.bloomberght.com/tahvil/tr-10-yillik-tahvil")[0]['SON'][10] / 10000

[*********************100%***********************]  1 of 1 completed


C:\Users\emir.e\AppData\Local\Temp\ipykernel_20780\1769103818.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = data['Adj Close'][-1]


In [287]:
BlackScholes(rf, current_price, 48, 1, 0.7, "C", 0.02)

Call Option Price: 13.57857
-----------------
Call Delta: 0.70294
Gamma: 0.01147
Vega: 0.14918
Call Theta: -0.03467
Call Rho: 0.16118
